In [27]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


# Eksperyment A/B
Najpierw zbierane są dane do przeprowadzenia eksperymentu a następnie jest on analizowany. 

## Zebranie danych do testu
*Zdefiniowanie pliku w któym zostaną zapisane eksperymenty oraz ścieżek do danych testowych*

In [2]:
PATH_TO_DATA_X_TEST = 'content/custom_data/X_test.csv'
PATH_TO_DATA_Y_TEST = 'content/custom_data/Y_test.csv'
PATH_TO_EXPERIMENT_DATA = 'content/results/ab_results.json'

*Załadowanie bibliotek, modułów i plików ze zbiorami testowymi*

In [3]:
import requests
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, f1_score
from scipy import stats
import json
from numpy import var
import matplotlib.pyplot as plt

from model_creation import load_and_preprocess_data

all_metrics = []

X_test = load_and_preprocess_data(PATH_TO_DATA_X_TEST)
Y_test = load_and_preprocess_data(PATH_TO_DATA_Y_TEST)

def plot_pvalue_changes(base, advanced):
	shapiro_p_values = []
	levene_p_values = []
	ttest_p_values = []

	for i in range(3, len(base) + 1):
		base_sample = base[:i]
		advanced_sample = advanced[:i]

		_, shapiro_p_base = stats.shapiro(base_sample)
		_, shapiro_p_adv = stats.shapiro(advanced_sample)
		shapiro_p_values.append((shapiro_p_base, shapiro_p_adv))

		_, levene_p = stats.levene(base_sample, advanced_sample)
		levene_p_values.append(levene_p)

		_, ttest_p = stats.ttest_ind(base_sample, advanced_sample, equal_var=True)
		ttest_p_values.append(ttest_p)

	x = range(3, len(base) + 1)
	shapiro_base_p, shapiro_adv_p = zip(*shapiro_p_values)

	plt.figure(figsize=(12, 8))

	plt.subplot(3, 1, 1)
	plt.plot(x, shapiro_base_p, label='Shapiro-Wilk Base', marker='o')
	plt.plot(x, shapiro_adv_p, label='Shapiro-Wilk Advanced', marker='o')
	plt.axhline(y=0.05, color='r', linestyle='--', label='p = 0.05')
	plt.title('Shapiro-Wilk Test p-values')
	plt.xlabel('Number of Samples')
	plt.ylabel('p-value')
	plt.legend()

	plt.subplot(3, 1, 2)
	plt.plot(x, levene_p_values, label='Levene Test', color='g', marker='o')
	plt.axhline(y=0.05, color='r', linestyle='--', label='p = 0.05')
	plt.title('Levene Test p-values')
	plt.xlabel('Number of Samples')
	plt.ylabel('p-value')
	plt.legend()

	plt.subplot(3, 1, 3)
	plt.plot(x, ttest_p_values, label='t-Test', color='b', marker='o')
	plt.axhline(y=0.05, color='r', linestyle='--', label='p = 0.05')
	plt.title('t-Test p-values')
	plt.xlabel('Number of Samples')
	plt.ylabel('p-value')
	plt.legend()

	plt.tight_layout()
	plt.show()

Do uzyskania miarodajnych wyników oraz wykrycia małych różnic w modelach wymagane są duże ilości danych. Ze względu na ograniczenia w ilości danych, spowodowane tym, że test przeprowadzono na podstawie zebranych danych, a nie na danych produkcyjnych, zastosowano symulację większej ilości danych. Dokonano tego poprzez podział danych na podstawie "seedów" i kilkunastokrotne wykorzystanie tych samych danych, lecz podzielonych w różny sposób.

*Wybranie seedów, na podstawie których zostały podzielone przypadki między model A i B.*

In [8]:
seeds = [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]

*Sprawdzenie, czy mikroserwis jest włączony*

In [15]:
try:
    requests.get("http://localhost:8080/predict")
    print("Serwis działa")
except requests.exceptions.RequestException as e:
    print("Mikroserwis nie działa lub nie jest włączony")
    raise e

Mikroserwis nie działa lub nie jest włączony


ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002DB60961F10>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

*Zebranie predykcji dla podzielonych danych (połowa danych przewidywana jest przez model bazowy a połowa przez model zaawansowany) i zapisanie metryk jakości modeli na ich podstawie*

In [ ]:
for seed in seeds:
    print(f"Running test for seed: {seed}")

    X_base, X_advanced, Y_base, Y_advanced = train_test_split(X_test, Y_test, test_size=0.5, random_state=seed)
    base_dicts = X_base.to_dict(orient='records')
    advanced_dicts = X_advanced.to_dict(orient='records')

    # Make predictions for divided sets
    base_predictions = []
    for entry in tqdm(base_dicts, desc="Sending requests for predictions (Base)"):
        response = requests.post("http://localhost:8080/predict_base", json=entry)
        base_predictions.append(response.json()['base_prediction'])

    advanced_predictions = []
    for entry in tqdm(advanced_dicts, desc="Sending requests for predictions (Advanced)"):
        response = requests.post("http://localhost:8080/predict_advanced", json=entry)
        advanced_predictions.append(response.json()['advanced_prediction'])

    # Append predictions to the Y data
    base_results = Y_base
    base_results["base_predictions"] = base_predictions
    advanced_results = Y_advanced
    advanced_results["advanced_predictions"] = advanced_predictions
    
    # For base model
    base_confusion_matrix = confusion_matrix(base_results['premium_user'], base_results['base_predictions'])
    base_precision = precision_score(base_results["premium_user"], base_results["base_predictions"])
    base_accuracy = accuracy_score(base_results["premium_user"], base_results["base_predictions"])
    base_f1 = f1_score(base_results["premium_user"], base_results["base_predictions"])
    
    # For advanced model
    advanced_confusion_matrix = confusion_matrix(advanced_results['premium_user'], advanced_results['advanced_predictions'])
    advanced_precision = precision_score(advanced_results["premium_user"], advanced_results["advanced_predictions"])
    advanced_accuracy = accuracy_score(advanced_results["premium_user"], advanced_results["advanced_predictions"])
    advanced_f1 = f1_score(advanced_results["premium_user"], advanced_results["advanced_predictions"])

    # Collect metrics to dictionary
    all_metrics.append({
    "seed": seed,
    "base": {
        "confusion_matrix": base_confusion_matrix.tolist(),
        "precision": round(base_precision * 100, 3),
        "accuracy": round(base_accuracy * 100, 3),
        "f1_score": round(base_f1 * 100, 3)
    },
    "advanced": {
        "confusion_matrix": advanced_confusion_matrix.tolist(),
        "precision": round(advanced_precision * 100, 3),
        "accuracy": round(advanced_accuracy * 100, 3),
        "f1_score": round(advanced_f1 * 100, 3)
    }
})

Running test for seed: 16


Sending requests for predictions (Base):   0%|          | 0/150 [00:04<?, ?it/s]


KeyboardInterrupt: 

*Zapisanie otrzymanych metryk w pliku json*

In [ ]:
try:
    with open(PATH_TO_EXPERIMENT_DATA, 'r') as output_file:
        loaded_metrices = json.load(output_file)
except FileNotFoundError:
    loaded_metrices = []
    
loaded_metrices.extend(all_metrics)
    
try:
    with open(PATH_TO_EXPERIMENT_DATA, 'w') as output_file:
        json.dump(loaded_metrices, output_file, indent=4)
except Exception as e:
    print(f"Error: {e}")

## Test t-Studenta
**Hipoteza zerowa (H0)** - Nie ma statystycznej różnicy między modelem zaawansowanym (las losowy) a bazowym (regresja logistyczna).

**Hipoteza alternatywna (H1)** - Istnieje statystyczna różnica między modelem zaawansowanym (las losowy) a bazowym (regresja logistyczna).

Jeśli p-value jest mniejsza od przyjętego alfa (standardowo 0.05), to znaczy, że wynik jest statystycznie istotny i możemy odrzucić hipotezę zerową.

Dodatkowo jeśli wartość testu t-Studenta jest ujemna to znaczy, że model bazowy uzyskuje średnio gorsze wyniki od modelu zaawansowanego.

*Załadowanie zebranych danych, niżej jest wczytanie danych z pliku*

In [ ]:
base_precisions = []
advanced_precisions = []

base_accuracies = []
advanced_accuracies = []

base_f1_scores = []
advanced_f1_scores = []
    
for experiment in all_metrics:
    base_precisions.append(experiment['base']['precision'])
    advanced_precisions.append(experiment['advanced']['precision'])

    base_accuracies.append(experiment['base']['accuracy'])
    advanced_accuracies.append(experiment['advanced']['accuracy'])

    base_f1_scores.append(experiment['base']['f1_score'])
    advanced_f1_scores.append(experiment['advanced']['f1_score'])

*Opcjonalne wczytanie danych z pliku*

In [ ]:
NUMBER_OF_SAMPLES = 10

try:
    with open(PATH_TO_EXPERIMENT_DATA, 'r') as input_file:
        loaded_metrices = json.load(input_file)
except Exception as e:
    print(f"Error: {e}")
    raise e

base_precisions = []
advanced_precisions = []

base_accuracies = []
advanced_accuracies = []

base_f1_scores = []
advanced_f1_scores = []
    
for experiment in loaded_metrices[:NUMBER_OF_SAMPLES]:
    base_precisions.append(experiment['base']['precision'])
    advanced_precisions.append(experiment['advanced']['precision'])

    base_accuracies.append(experiment['base']['accuracy'])
    advanced_accuracies.append(experiment['advanced']['accuracy'])

    base_f1_scores.append(experiment['base']['f1_score'])
    advanced_f1_scores.append(experiment['advanced']['f1_score'])


### Sprawdzenie warunków początkowych
**Warunki początkowe testu T-Studenta:**
- Próby A i B są niezależne - są niezależne ponieważ w każdej iteracji zbiory testowe są rozłączne
- Rozkład zmiennych musi być zgodny z rozkłądem normalnym - warunek sprawdzony zostanie za pomocą testu Shapiro-Wilka 
- Wariancje prób mają być podobne - zostanie to sprawdzone testem Levene'a

#### Sprawdzenie rozkładu prób za pomocą testu Shapiro-Wilka

In [33]:
shapiro_base_precision = stats.shapiro(base_precisions)
shapiro_advanced_precision = stats.shapiro(advanced_precisions)

shapiro_base_accuracy = stats.shapiro(base_accuracies)
shapiro_advanced_accuracy = stats.shapiro(advanced_accuracies)

shapiro_base_f1 = stats.shapiro(base_f1_scores)
shapiro_advanced_f1 = stats.shapiro(advanced_f1_scores)

if shapiro_base_precision.pvalue >= 0.05:
	print("Nie ma powodu do odrzucenia hipotezy zerowej mówiącej, że rozkład precyzji modelu bazowego jest normalny.")
else:
	print("Istnieją podstawy do twierdzenia, że rozkład precyzji modelu bazowego nie jest normalny.")
if shapiro_advanced_precision.pvalue >= 0.05:
	print("Nie ma powodu do odrzucenia hipotezy zerowej mówiącej, że rozkład precyzji modelu zaawansowanego jest normalny.")
else:
	print("Istnieją podstawy do twierdzenia, że rozkład precyzji modelu zaawansowanego nie jest normalny.")

if shapiro_base_accuracy.pvalue >= 0.05:
	print("Nie ma powodu do odrzucenia hipotezy zerowej mówiącej, że rozkład dokładności modelu bazowego jest normalny.")
else:
	print("Istnieją podstawy do twierdzenia, że rozkład dokładności modelu bazowego nie jest normalny.")
if shapiro_advanced_accuracy.pvalue >= 0.05:
	print("Nie ma powodu do odrzucenia hipotezy zerowej mówiącej, że rozkład dokładności modelu zaawansowanego jest normalny.")
else:
	print("Istnieją podstawy do twierdzenia, że rozkład dokładności modelu zaawansowanego nie jest normalny.")

if shapiro_base_f1.pvalue >= 0.05:
	print("Nie ma powodu do odrzucenia hipotezy zerowej mówiącej, że rozkład F1-Score modelu bazowego jest normalny.")
else:
	print("Istnieją podstawy do twierdzenia, że rozkład F1-Score modelu bazowego nie jest normalny.")
if shapiro_advanced_f1.pvalue >= 0.05:
	print("Nie ma powodu do odrzucenia hipotezy zerowej mówiącej, że rozkład F1-Score modelu zaawansowanego jest normalny.")
else:
	print("Istnieją podstawy do twierdzenia, że rozkład F1-Score modelu zaawansowanego nie jest normalny.")


Istnieją podstawy do twierdzenia, że rozkład precyzji modelu bazowego nie jest normalny.
Istnieją podstawy do twierdzenia, że rozkład precyzji modelu zaawansowanego nie jest normalny.
Istnieją podstawy do twierdzenia, że rozkład dokładności modelu bazowego nie jest normalny.
Istnieją podstawy do twierdzenia, że rozkład dokładności modelu zaawansowanego nie jest normalny.
Istnieją podstawy do twierdzenia, że rozkład F1-Score modelu bazowego nie jest normalny.
Istnieją podstawy do twierdzenia, że rozkład F1-Score modelu zaawansowanego nie jest normalny.


C:\Users\Atitude\AppData\Local\Temp\ipykernel_1104\4112777212.py:1: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  shapiro_base_precision = stats.shapiro(base_precisions)
C:\Users\Atitude\AppData\Local\Temp\ipykernel_1104\4112777212.py:2: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  shapiro_advanced_precision = stats.shapiro(advanced_precisions)
C:\Users\Atitude\AppData\Local\Temp\ipykernel_1104\4112777212.py:4: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  shapiro_base_accuracy = stats.shapiro(base_accuracies)
C:\Users\Atitude\AppData\Local\Temp\ipykernel_1104\4112777212.py:5: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation f

Okazało się, że precyzje nie mają rozkładu normalnego, więc nie powinno się ich używać w teście t-Studenta. Dokładności i F1-Scores nadają się do tej analizy.

#### Sprawdzenie wariancji prób testem Levene'a

In [34]:
precision_levene, precision_p = stats.levene(base_precisions, advanced_precisions)

accuracy_levene, accuracy_p = stats.levene(base_accuracies, advanced_accuracies)

f1_levene, f1_p = stats.levene(base_f1_scores, advanced_f1_scores)

if precision_p >= 0.05:
	print("Nie ma powodu do odrzucenia hipotezy zerowej mówiącej, że wariancje precyzji modelu bazowego i zaawansowanego są podobne.")
else:
	print("Istnieją podstawy do twierdzenia, że wariancje precyzji modelu bazowego i zaawansowanego nie są podobne.")

if accuracy_p >= 0.05:
	print("Nie ma powodu do odrzucenia hipotezy zerowej mówiącej, że wariancje dokładności modelu bazowego i zaawansowanego są podobne.")
else:
	print("Istnieją podstawy do twierdzenia, że wariancje dokładności modelu bazowego i zaawansowanego nie są podobne.")

if f1_p >= 0.05:
	print("Nie ma powodu do odrzucenia hipotezy zerowej mówiącej, że wariancje F1-Score modelu bazowego i zaawansowanego są podobne.")
else:
	print("Istnieją podstawy do twierdzenia, że wariancje F1-Score modelu bazowego i zaawansowanego nie są podobne.")


Istnieją podstawy do twierdzenia, że wariancje precyzji modelu bazowego i zaawansowanego nie są podobne.
Istnieją podstawy do twierdzenia, że wariancje dokładności modelu bazowego i zaawansowanego nie są podobne.
Istnieją podstawy do twierdzenia, że wariancje F1-Score modelu bazowego i zaawansowanego nie są podobne.


d:\SyncVestigat\2_Studia\5 sem\IUM\gitlaby\git_vpo_dok_wstep\ium-projekt-mast-have-24z\.venv\Lib\site-packages\scipy\stats\_morestats.py:3361: RuntimeWarning: invalid value encountered in scalar divide
  W = numer / denom


Wariancje precyzji, dokładności i F1-Score są podobne dla modelu bazowego i zaawansowanego.

### Przeprowadzenie testu T-studenta

In [35]:
alpha = 0.05

t_stat_precision, p_value_precision = stats.ttest_ind(advanced_precisions, base_precisions)
t_stat_accuracy, p_value_accuracy = stats.ttest_ind(advanced_accuracies, base_accuracies)
t_stat_f1, p_value_f1 = stats.ttest_ind(advanced_f1_scores, base_f1_scores)

print(f"Test t-Studenta dla precyzji: t-statistic = {t_stat_precision}, p-value = {p_value_precision}")
print(f"Test t-Studenta dla dokładności: t-statistic = {t_stat_accuracy}, p-value = {p_value_accuracy}")
print(f"Test t-Studenta dla F1-Score: t-statistic = {t_stat_f1}, p-value = {p_value_f1}")
print("")
if p_value_precision < alpha:
    print("Różnica w precyzji jest statystycznie istotna")
    if t_stat_precision > 0:
        print("\tModel zaawansowany ma lepszą precyzję")
    elif t_stat_precision < 0:
        print("\tModel zaawansowany ma gorszą precyzję")
else:
    print("Różnica w precyzji nie jest statystycznie istotna")
print("")
if p_value_accuracy < alpha:
    print("Różnica w dokładności jest statystycznie istotna")
    if t_stat_accuracy > 0:
        print("\tModel zaawansowany ma lepszą dokładność")
    elif t_stat_accuracy < 0:
        print("\tModel zaawansowany ma gorszą dokładność")
else:
    print("Różnica w dokładności nie jest statystycznie istotna")
print("")

if p_value_f1 < alpha:
    print("Różnica w F1-Score jest statystycznie istotna")
    if t_stat_f1 > 0:
        print("\tModel zaawansowany ma lepszy F1-Score")
    elif t_stat_f1 < 0:
        print("\tModel zaawansowany ma gorszy F1-Score")
else:
    print("Różnica w F1-Score nie jest statystycznie istotna")

Test t-Studenta dla precyzji: t-statistic = nan, p-value = nan
Test t-Studenta dla dokładności: t-statistic = nan, p-value = nan
Test t-Studenta dla F1-Score: t-statistic = nan, p-value = nan

Różnica w precyzji nie jest statystycznie istotna

Różnica w dokładności nie jest statystycznie istotna

Różnica w F1-Score nie jest statystycznie istotna


d:\SyncVestigat\2_Studia\5 sem\IUM\gitlaby\git_vpo_dok_wstep\ium-projekt-mast-have-24z\.venv\Lib\site-packages\scipy\stats\_stats_py.py:6558: RuntimeWarning: invalid value encountered in scalar divide
  svar = ((n1 - 1) * v1 + (n2 - 1) * v2) / df


Tak zmieniały się p-values przy dobieraniu kolejnych próbek

In [ ]:
plot_pvalue_changes(base_precisions, advanced_precisions)
plot_pvalue_changes(base_accuracies, advanced_accuracies)
plot_pvalue_changes(base_accuracies, advanced_accuracies)

Wystarczy 5 iteracji eksperymentu A/B, by stwierdzić statystycznie istotną różnicę w dokładności i F1-Score między modelem bazowym a zaawansowanym. Nie uda się to w przypadku precyzji, nawet przy 30 iteracjach.